<br>

# Introdução

In [ ]:
#!pip3 install simplekml --upgrade
#!pip3 install open-geodata --upgrade
#!pip3 install traquitanas --upgrade

In [ ]:
import os
import folium
import simplekml
import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
from folium import plugins

from shapely.geometry import Point
from datetime import date

In [ ]:
from open_geodata import geo
from traquitanas.geo import layers as tt

In [ ]:
from paths import *
from cod_ibge import *

<br>

## Municípios

Lista quais os Municípios que exportei dados.

In [ ]:
os.listdir(output_path_cidades)

In [ ]:
# Parameters
estado = 'SP'
cod_ibge = '3548906' # São Carlos

# Adjust Code
cod_ibge_adjusted = adjust_cod_ibge(cod_ibge)

# Path name
city_path = '{}_{}'.format(estado, cod_ibge_adjusted)
city_path

In [ ]:
# Confere se o município selecionado tem dados exportados
city_path in os.listdir(output_path_cidades)

In [ ]:
# Cria Diretórios de output de Mapas
geo_path = os.path.join(output_path_cidades, city_path, 'analysis_geo')
os.makedirs(geo_path, exist_ok=True)

In [ ]:
# Qual é esse ano?!
this_year = date.today().year
this_year

<br>

# Geodata

In [ ]:
# List geodataframes
geo.get_dataset_names()

# Load geodataframe
gdf_sp = geo.load_dataset('sp_250k_wgs84.geojson')
gdf_sp

<br>

# Folium

<br>

## Layer: Captação

In [ ]:
## Lê Dados de Captação

# Read Table
df = pd.read_excel(
    os.path.join(output_path_cidades, city_path, 'cadastro', 'cadastro_pontos_captacao.xlsx')
)

# Filtra os Últimos
if this_year in set(df['Ano De Referência']):
    df = df.loc[df['Ano De Referência'] == this_year]
    df.reset_index(drop=True, inplace=True)
    display(df.head())
else:
    print('d')

In [ ]:
# dtypes
#print(df.dtypes)

# Adjusta os dtypes
df['Longitude'] = pd.to_numeric(df['Longitude'].str.replace(',', '.'), errors='coerce')
df['Latitude'] = pd.to_numeric(df['Latitude'].str.replace(',', '.'), errors='coerce')

In [ ]:
# Create Geometry
geometry = [Point(xy) for xy in zip(df['Longitude'], df['Latitude'])]

# Create Geodataframe
gdf = gpd.GeoDataFrame(
    df,
    crs='EPSG:4326',
    geometry=geometry
)

gdf.to_file(
    os.path.join(geo_path, 'cadastro_pontos_captacao.geojson'),
    driver='GeoJSON'
)

In [ ]:
def add_lyr_captacao(output_geo_path):
    # Input
    gdf = gpd.read_file(os.path.join(output_geo_path, 'cadastro_pontos_captacao.geojson'))
    gdf['geometry'] = gdf['geometry'].replace('None', np.nan)
    gdf = gdf.loc[gdf['geometry'] != None]
    gdf = gdf.to_crs(epsg=4326)
    
    # Popup
    gdf['popup'] = gdf.apply(popup_html_est_aut_empresas, axis=1)

    # Layer
    lyr = folium.GeoJson(
        gdf,
        name='Captação para Abastecimento',
        smooth_factor=1.0,
        style_function=lambda x: {
            'fillColor': '#DC143C',
            'color': '#DC143C',
            'weight': 1,
            'fillOpacity': 0.3,
        },
        highlight_function=lambda x: {
            'weight': 3,
            'fillOpacity': 0.6,
        },
        popup=folium.GeoJsonPopup(
            ['popup'],
            parse_html=False,
            max_width='400',
            show=False,
            labels=False,
            sticky=True,            
        ),
        marker=folium.Marker(
            icon=folium.Icon(
                color='lightgray',
                icon_color='#FFFF00',
                #icon='leaf',
            ),
        ),
        tooltip=folium.GeoJsonTooltip(
            fields=['Tipo Da Forma De Abastecimento'],
            aliases=['Forma'],
            sticky=True,
            opacity=0.9,
            direction='right',
        ),
        embed=False,
        zoom_on_click=False,
        control=True,
        show=True,
    )
    return lyr


# PopUp
def popup_html_est_aut_empresas(row):
    # Data
    nome = row['Nome Da Instiuição']
    tipo = row['Tipo De Captação']
    outorga = row['Outorga']
    tipo_forma = row['Tipo Da Forma De Abastecimento']
    nome_forma = row['Nome Da Forma De Abastecimento']
	
    # Infos
    popup = """
    <div>
    <h5>{}</h5>
    <br>Responsável: <b>{}</b>
    <br>Tipo da Forma de Abastecimento: <b>{}</b>
    <br>Nome da Forma de Abastecimento: <b>{}</b>
    <br>Tem Ouutorga?: <b>{}</b>
    </div>
    """.format(tipo_forma, nome, tipo, nome_forma, outorga)
    return popup

<br>

## Layer: Município

In [ ]:
gdf_sp_mun = gdf_sp.loc[gdf_sp['id_municipio'] == int(cod_ibge)]
gdf_sp_mun.plot()
gdf_sp_mun.to_file(
    os.path.join(geo_path, 'limite_municipio.geojson'),
    driver='GeoJSON'
)

In [ ]:
def add_lyr_municipio(output_geo_path):
    # Input
    gdf = gpd.read_file(os.path.join(output_geo_path, 'limite_municipio.geojson'))
    gdf = gdf.to_crs(epsg=4326)
    
    # Popup
    #gdf['popup'] = gdf.apply(popup_html_est_aut_empresas, axis=1)

    # Layer
    lyr = folium.GeoJson(
        gdf,
        name='Limite Municipal',
        smooth_factor=1.0,
        style_function=lambda x: {
            'fillColor': '#DC143C',
            'color': '#DC143C',
            'weight': 1,
            'fillOpacity': 0.3,
        },
        highlight_function=lambda x: {
            'weight': 3,
            'fillOpacity': 0.6,
        },
        # popup=folium.GeoJsonPopup(
        #     ['popup'],
        #     parse_html=False,
        #     max_width='400',
        #     show=False,
        #     labels=False,
        #     sticky=True,            
        # ),
        marker=folium.Marker(
            icon=folium.Icon(
                color='lightgray',
                icon_color='#FFFF00',
                #icon='leaf',
            ),
        ),
        # tooltip=folium.GeoJsonTooltip(
        #     fields=['municipio_nome'],
        #     aliases=['Município'],
        #     sticky=True,
        #     opacity=0.9,
        #     direction='right',
        # ),
        embed=False,
        zoom_on_click=False,
        control=True,
        show=True,
    )
    return lyr

<br>

## Layer: Municípios Vizinhos

In [ ]:
gdf_sp_viz = geo.share_boundary(gdf_sp, gdf_sp_mun)
gdf_sp_viz.to_file(
    os.path.join(geo_path, 'limite_municipios_vizinhos.geojson'),
    driver='GeoJSON'
)
gdf_sp_viz.plot()

In [ ]:
def add_lyr_municipios_vizinhos(output_geo_path):
    # Input
    gdf = gpd.read_file(os.path.join(output_geo_path, 'limite_municipios_vizinhos.geojson'))
    gdf = gdf.to_crs(epsg=4326)    
    
    # Popup
    #gdf['popup'] = gdf.apply(popup_html_est_aut_empresas, axis=1)

    # Layer
    lyr = folium.GeoJson(
        gdf,
        name='Limite Municipíos Vizinhos',
        smooth_factor=1.0,
        style_function=lambda x: {
            'fillColor': '#DC143C',
            'color': '#DC143C',
            'weight': 1,
            'fillOpacity': 0.3,
        },
        highlight_function=lambda x: {
            'weight': 3,
            'fillOpacity': 0.6,
        },
        # popup=folium.GeoJsonPopup(
        #     ['popup'],
        #     parse_html=False,
        #     max_width='400',
        #     show=False,
        #     labels=False,
        #     sticky=True,            
        # ),
        marker=folium.Marker(
            icon=folium.Icon(
                color='lightgray',
                icon_color='#FFFF00',
                #icon='leaf',
            ),
        ),
        tooltip=folium.GeoJsonTooltip(
            fields=['municipio_nome'],
            aliases=['Município'],
            sticky=True,
            opacity=0.9,
            direction='right',
        ),
        embed=False,
        zoom_on_click=False,
        control=True,
        show=True,
    )
    return lyr

<br>

## Map

In [ ]:
def get_map(input_geojson):
    # Input
    gdf = gpd.read_file(input_geojson)
    gdf = gdf.to_crs(epsg=4326)
    sw = gdf.bounds[['miny', 'minx']].min().values.tolist()
    ne = gdf.bounds[['maxy', 'maxx']].max().values.tolist()
    bounds = [sw, ne]
    
    # Zoom
    min_zoom = 8
    max_zoom = 21
    padding = 1
    
    # Create Map
    m = folium.Map(
        min_zoom=min_zoom,
        max_zoom=max_zoom,
        max_bounds=True,
        min_lat=bounds[0][0]*((100+padding)/100),
        min_lon=bounds[0][1]*((100+padding)/100),
        max_lat=bounds[1][0]*((100-padding)/100),
        max_lon=bounds[1][1]*((100-padding)/100),
        tiles=None,
        # zoom_delta=0.1,
        # zoom_start=10,
    )
    
    # Add Layers
    m.add_child(tt.add_lyr_google_hybrid(min_zoom, max_zoom))
    m.add_child(tt.add_lyr_google_satellite(min_zoom, max_zoom))
    m.add_child(tt.add_lyr_google_terrain(min_zoom, max_zoom))    
    m.add_child(tt.add_lyr_google_streets(min_zoom, max_zoom))    

    # Monitoramento
    m.add_child(add_lyr_captacao(geo_path))
    m.add_child(add_lyr_municipio(geo_path))
    m.add_child(add_lyr_municipios_vizinhos(geo_path))    

    # Plugins
    m.fit_bounds(bounds)
    plugins.Fullscreen(
        position='topleft',
        title='Clique para Maximizar',
        title_cancel='Mininizar',
    ).add_to(m)
    folium.LayerControl(
        position='topright',
        collapsed=False,
    ).add_to(m)
    return m

In [ ]:
# Create Maps
m = get_map(os.path.join(geo_path, 'cadastro_pontos_captacao.geojson'))

# Save
m.save(os.path.join(geo_path, 'map_captacao.html'))

# Results
m